**ETL**

In [82]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [83]:
import csv

In [84]:
data = pd.read_csv("movies_dataset.csv")

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\133786677.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("movies_dataset.csv")


**PRIMERA PARTE: DASANIDAR COLUMNAS**

Comenzamos con la columna `belongs_to_collection` (Diccionario), en cada registro en caso de no ser nulo, se reemplazará solo con el nombre

In [85]:
for i in range(len(data)):
    try:
        if isinstance(eval(data["belongs_to_collection"][i]), dict):
            data["belongs_to_collection"][i] = eval(data["belongs_to_collection"][i])["name"]
    except:
        pass

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\2878617107.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["belongs_to_collection"][i] = eval(data["belongs_to_collection"][i])["name"]


Para `genres` se añadirán 3 columnas más al DataFrame que serán `g1`, `g2`, `g3` y se agregaran los generos a cada columna respectivamente

In [86]:
pd.json_normalize(eval(data["genres"][2]))["name"][0]

'Romance'

Se desanida la columna genres (Diccionario)

In [87]:
data['g1'] = np.nan
data['g2'] = np.nan
data['g3'] = np.nan


for i in range(len(data)):
    try:
      df = pd.json_normalize(eval(data['genres'][i]))

      for j in range(len(df)):
        data[f'g{j+1}'][i] = df['name'][j]
    except:
        pass

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\2090772770.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'g{j+1}'][i] = df['name'][j]


Desnidamos la columna data `production_companies` (Lista)

In [88]:
data['produc_com1'] = np.nan
data['produc_com2'] = np.nan

for i in range(len(data)):
    try:
        df = pd.json_normalize(eval(data['production_companies'][i]))

        for j in range(len(df)):
            data[f'produc_com{j+1}'][i] = df['name'][j]
    except:
        continue

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\3212550824.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'produc_com{j+1}'][i] = df['name'][j]
C:\Users\facun\AppData\Local\Temp\ipykernel_13068\3212550824.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'produc_com{j+1}'][i] = df['name'][j]


Se desanidará la columna `production_countries` (Lista)

In [89]:
data['production_countries'][34556]

"[{'iso_3166_1': 'IT', 'name': 'Italy'}]"

In [90]:
data['country'] = np.nan

for i in range(len(data)):
    try:
        df = pd.json_normalize(eval(data['production_countries'][i]))

        for j in range(len(df)):
            data['country'][i] = df['name'][j]
    except:
        continue

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\2701637996.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['country'][i] = df['name'][j]


Se desanidará la columna `spoken_languages`

In [91]:
data['language'] = np.nan

for i in range(len(data)):
    try:
        df = pd.json_normalize(eval(data['spoken_languages'][i]))

        for j in range(len(df)):
            data['language'][i] = df['name'][j]
    except:
        continue

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\640928165.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['language'][i] = df['name'][j]


In [92]:
columnas = ['genres', 'production_companies',
       'production_countries','spoken_languages']

data = data.drop(columns=columnas)

In [93]:
data

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,video,vote_average,vote_count,g1,g2,g3,produc_com1,produc_com2,country,language
0,False,Toy Story Collection,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,False,7.7,5415.0,Animation,Comedy,Family,Pixar Animation Studios,NaN,United States of America,English
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,False,6.9,2413.0,Adventure,Fantasy,Family,TriStar Pictures,Teitler Film,United States of America,Français
2,False,Grumpy Old Men Collection,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,False,6.5,92.0,Romance,Comedy,NaN,Warner Bros.,Lancaster Gate,United States of America,English
3,False,NaN,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,False,6.1,34.0,Comedy,Drama,Romance,Twentieth Century Fox Film Corporation,NaN,United States of America,English
4,False,Father of the Bride Collection,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,False,5.7,173.0,Comedy,NaN,NaN,Sandollar Productions,Touchstone Pictures,United States of America,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,...,False,4.0,1.0,Drama,Family,NaN,NaN,NaN,Iran,فارسی
45462,False,NaN,0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,...,False,9.0,3.0,Drama,NaN,NaN,Sine Olivia,NaN,Philippines,
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,False,3.8,6.0,Action,Drama,Thriller,American World Pictures,NaN,United States of America,English
45464,False,NaN,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,...,False,0.0,0.0,NaN,NaN,NaN,Yermoliev,NaN,Russia,NaN


Los valores nulos de los campos `revenue`, `budget` deben ser rellenados por el número 0

In [94]:
data['budget'] = data['budget'].fillna(0)
data['revenue'] = data['revenue'].fillna(0)

Los valores nulos del campo `release date` deben ser eliminados

In [95]:
nombre_columna = 'release_date'
data = data.dropna(subset=[nombre_columna])

 De haber fechas, deberán tener el formato `AAAA-mm-dd`, además deberán crear la columna `release_year` donde extraerán el año de la fecha de estreno.


In [96]:
data['release_date'] =  pd.to_datetime(data['release_date'], errors='coerce')

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\4179229598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['release_date'] =  pd.to_datetime(data['release_date'], errors='coerce')


In [97]:
data['release_year'] = data['release_date'].dt.strftime('%Y')

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\256904507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['release_year'] = data['release_date'].dt.strftime('%Y')


Acá transformamos las columnas `release_date` y `release_year` de formato Object a Date


In [98]:
data['release_year'] = pd.to_datetime(data['release_date'], format='%Y')

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\1601711881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['release_year'] = pd.to_datetime(data['release_date'], format='%Y')


In [99]:
data['release_year'] = data['release_year'].dt.to_period('Y')

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\3553755221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['release_year'] = data['release_year'].dt.to_period('Y')


Crear la columna con el retorno de inversión, llamada `return` con los campos `revenue` y `budget`, dividiendo estas dos últimas `revenue/budget`, cuando no hay datos disponibles para calcularlo, deberá tomar el valor `0`.


Acá se noto unos datos extraños por lo que se decidió reemplazarlos por cero para poder hacer el return correctamente

In [100]:
data['budget'] = data['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0)
data['budget'] = data['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0)
data['budget'] = data['budget'].replace('/ff9qCepilowshEtG2GYWwzt2bs4.jpg', 0)
data['budget'] = data['budget'].astype(int)

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\118796259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['budget'] = data['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0)
C:\Users\facun\AppData\Local\Temp\ipykernel_13068\118796259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['budget'] = data['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0)
C:\Users\facun\AppData\Local\Temp\ipykernel_13068\118796259.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [101]:
def calcular_columna(row):
    if row['budget'] != 0 and row['revenue'] != 0:
        return row['revenue'] / row['budget']
    else:
        return np.nan
data['return'] = data.apply(calcular_columna, axis=1)

C:\Users\facun\AppData\Local\Temp\ipykernel_13068\3988146083.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['return'] = data.apply(calcular_columna, axis=1)


Eliminar las columnas que no serán utilizadas: `video`,`imdb_id`,`adult`,`original_title`,`poster_path` y `homepage`.


In [102]:
columnas = ['video', 'imdb_id', 'adult', 'original_title',
            'poster_path', 'homepage']

data = data.drop(columns=columnas)

In [103]:
data

,belongs_to_collection,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,...,vote_count,g1,g2,g3,produc_com1,produc_com2,country,language,release_year,return
0,Toy Story Collection,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,...,5415.0,Animation,Comedy,Family,Pixar Animation Studios,NaN,United States of America,English,1995,12.451801
1,NaN,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,...,2413.0,Adventure,Fantasy,Family,TriStar Pictures,Teitler Film,United States of America,Français,1995,4.043035
2,Grumpy Old Men Collection,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,...,92.0,Romance,Comedy,NaN,Warner Bros.,Lancaster Gate,United States of America,English,1995,NaN
3,NaN,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,...,34.0,Comedy,Drama,Romance,Twentieth Century Fox Film Corporation,NaN,United States of America,English,1995,5.090760
4,Father of the Bride Collection,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,...,173.0,Comedy,NaN,NaN,Sandollar Productions,Touchstone Pictures,United States of America,English,1995,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,NaN,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,...,26.0,Drama,Action,Romance,Westdeutscher Rundfunk (WDR),Working Title Films,United States of America,English,1991,NaN
45462,NaN,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,...,3.0,Drama,NaN,NaN,Sine Olivia,NaN,Philippines,,2011,NaN
45463,NaN,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,...,6.0,Action,Drama,Thriller,American World Pictures,NaN,United States of America,English,2003,NaN
45464,NaN,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,...,0.0,NaN,NaN,NaN,Yermoliev,NaN,Russia,NaN,1917,NaN


In [104]:
df = pd.read_csv("credits.csv")

In [105]:
df['crew'] = df['crew'].apply(eval)

In [106]:
data_list = []
for i in range(len(df)):
    data2 = pd.DataFrame(df.crew[i])
    data2['id'] = df.id[i]
    data_list.append(data2)

dfcast = pd.concat(data_list, ignore_index=True)

In [107]:
prueba = dfcast[dfcast['job'] =='Director']

In [108]:
final = prueba.drop(columns=['profile_path', 'credit_id', 'gender', 'department', 'job'])

In [109]:
final

,id,name
0,862,John Lasseter
109,8844,Joe Johnston
122,15602,Howard Deutch
126,31357,Forest Whitaker
141,11862,Charles Shyer
...,...,...
464291,439050,Hamid Nematollah
464300,111109,Lav Diaz
464306,67758,Mark L. Lester
464311,227506,Yakov Protazanov


In [118]:
print (data['id'].dtype)

object


In [117]:
print (df['id'].dtype)

object


In [112]:
df['id'] = df['id'].astype(str)

In [ ]:
data = data.merge(final, on='id')

In [ ]:
final.to_csv('df_f')

LOAD

In [120]:
df_f = pd.read_csv('df_f.csv')

In [121]:
df_f

,Unnamed: 0,belongs_to_collection,budget,id,original_language,overview,popularity,release_date,revenue,runtime,...,g1,g2,g3,produc_com1,produc_com2,country,language,release_year,return,name
0,0,Toy Story Collection,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,...,Animation,Comedy,Family,Pixar Animation Studios,NaN,United States of America,English,1995.0,12.451801,John Lasseter
1,1,NaN,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,...,Adventure,Fantasy,Family,TriStar Pictures,Teitler Film,United States of America,Français,1995.0,4.043035,Joe Johnston
2,2,Grumpy Old Men Collection,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,...,Romance,Comedy,NaN,Warner Bros.,Lancaster Gate,United States of America,English,1995.0,NaN,Howard Deutch
3,3,NaN,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,...,Comedy,Drama,Romance,Twentieth Century Fox Film Corporation,NaN,United States of America,English,1995.0,5.090760,Forest Whitaker
4,4,Father of the Bride Collection,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,...,Comedy,NaN,NaN,Sandollar Productions,Touchstone Pictures,United States of America,English,1995.0,NaN,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49073,49073,NaN,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,...,Drama,Action,Romance,Westdeutscher Rundfunk (WDR),Working Title Films,United States of America,English,1991.0,NaN,John Irvin
49074,49074,NaN,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,...,Drama,NaN,NaN,Sine Olivia,NaN,Philippines,NaN,2011.0,NaN,Lav Diaz
49075,49075,NaN,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,...,Action,Drama,Thriller,American World Pictures,NaN,United States of America,English,2003.0,NaN,Mark L. Lester
49076,49076,NaN,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,...,NaN,NaN,NaN,Yermoliev,NaN,Russia,NaN,1917.0,NaN,Yakov Protazanov


In [122]:
df_f.drop(['Unnamed: 0'], axis=1)

,belongs_to_collection,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,...,g1,g2,g3,produc_com1,produc_com2,country,language,release_year,return,name
0,Toy Story Collection,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,...,Animation,Comedy,Family,Pixar Animation Studios,NaN,United States of America,English,1995.0,12.451801,John Lasseter
1,NaN,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,...,Adventure,Fantasy,Family,TriStar Pictures,Teitler Film,United States of America,Français,1995.0,4.043035,Joe Johnston
2,Grumpy Old Men Collection,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,...,Romance,Comedy,NaN,Warner Bros.,Lancaster Gate,United States of America,English,1995.0,NaN,Howard Deutch
3,NaN,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,...,Comedy,Drama,Romance,Twentieth Century Fox Film Corporation,NaN,United States of America,English,1995.0,5.090760,Forest Whitaker
4,Father of the Bride Collection,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,...,Comedy,NaN,NaN,Sandollar Productions,Touchstone Pictures,United States of America,English,1995.0,NaN,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49073,NaN,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,...,Drama,Action,Romance,Westdeutscher Rundfunk (WDR),Working Title Films,United States of America,English,1991.0,NaN,John Irvin
49074,NaN,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,...,Drama,NaN,NaN,Sine Olivia,NaN,Philippines,NaN,2011.0,NaN,Lav Diaz
49075,NaN,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,...,Action,Drama,Thriller,American World Pictures,NaN,United States of America,English,2003.0,NaN,Mark L. Lester
49076,NaN,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,...,NaN,NaN,NaN,Yermoliev,NaN,Russia,NaN,1917.0,NaN,Yakov Protazanov
